# Trip advisor reviews sentiment classification [PyTorch]
* Multiclass sentiment classification of hotel's reviews.
* Reference notebook: <https://www.kaggle.com/code/emirkocak/in-depth-series-sentiment-analysis-w-transformers/notebook>
* Dataset: <https://www.kaggle.com/code/emirkocak/in-depth-series-sentiment-analysis-w-transformers/input>

## Import libraries

In [2]:
import os
import re
import string
import random
import warnings
from typing import Union, List
from dataclasses import dataclass

import nltk
import torch
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.metrics import classification_report
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

import giskard
from giskard import Dataset, Model

2023-05-19 13:44:12.676739: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Notebook-level settings

In [3]:
# Disable warnings prints.
warnings.filterwarnings('ignore')

# Download list of english stopwords.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mykytaalekseiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Define constants

In [4]:
# Constants.
STOP_WORDS = set(stopwords.words('english'))

PRETRAINED_WEIGHTS_NAME = "distilbert-base-uncased"

TEXT_COLUMN_NAME = "Review"
TARGET_COLUMN_NAME = "label"

RANDOM_SEED = 0

# Paths.
DATA_PATH = os.path.join(".", "datasets", "trip_advisor_sentiment_classification_dataset", "tripadvisor_hotel_reviews.csv")

## Set random seeds

In [5]:
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

## Load data

In [6]:
def create_label(x: int) -> int:
    """Map rating to the label."""
    if x in [1, 2]:
        return 0
    if x == 3:
        return 1
    if x in [4, 5]:
        return 2

In [7]:
def load_data(**kwargs) -> pd.DataFrame:
    """Load data."""
    df = pd.read_csv(DATA_PATH, **kwargs)

    # Obtain labels for our task.
    df[TARGET_COLUMN_NAME] = df.Rating.apply(lambda x: create_label(x))
    df.drop(columns="Rating", inplace=True)

    return df

reviews_df = load_data(nrows=2000)
reviews_df.head()

,Review,label
0,nice hotel expensive parking got good deal sta...,2
1,ok nothing special charge diamond member hilto...,0
2,nice rooms not 4* experience hotel monaco seat...,1
3,"unique, great stay, wonderful time hotel monac...",2
4,"great stay great stay, went seahawk game aweso...",2


## Wrap dataset

In [8]:
wrapped_data = Dataset(reviews_df,
                       name="trip_advisor_reviews_sentiment",
                       target=TARGET_COLUMN_NAME,
                       column_types={TEXT_COLUMN_NAME: "text"})

Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."


## Data preprocessing

In [9]:
class CleanText:
    """Helper class to preprocess review's text."""
    def __init__(self, clean_pattern: str = r"[^A-ZĞÜŞİÖÇIa-zğüı'şöç0-9.\"',()]"):
        """Constructor of the class."""
        self.clean_pattern = clean_pattern

    def __call__(self, text: Union[str, list]) -> List[List[str]]:
        """Perform cleaning."""
        if isinstance(text, str):
            docs = [[text]]

        if isinstance(text, list):
            docs = text

        text = [[re.sub(self.clean_pattern, " ", sentence) for sentence in sentences] for sentences in docs]
        return text
    
def remove_emoji(data: str) -> str:
    """Remove emoji from the text."""
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoji, '', data)

regex = re.compile('[%s]' % re.escape(string.punctuation))
def remove_punctuation(text: str) -> str:
    """Remove punctuation from the text."""
    text = regex.sub(" ", text)
    return text

clean = CleanText()

In [10]:
def text_preprocessor(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess text."""
    # Remove emoji.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: remove_emoji(x))

    # Lower.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: x.lower())

    # Clean.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: clean(x)[0][0])

    # Remove punctuation.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: remove_punctuation(x))

    return df

reviews_df = text_preprocessor(reviews_df)

# Init Distill-BERT model

In [11]:
@dataclass
class Config:
    """Configuration of Distill-BERT model."""
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    batch_size = 128
    seq_length = 150
    add_special_tokens = True
    return_attention_mask = True
    pad_to_max_length = True
    return_tensors = 'pt'

In [12]:
# Load tokenizer.
tokenizer = DistilBertTokenizer.from_pretrained(PRETRAINED_WEIGHTS_NAME)

# Load model.
model = DistilBertForSequenceClassification.from_pretrained(PRETRAINED_WEIGHTS_NAME,
                                                            num_labels=3,
                                                            output_attentions=False,
                                                            output_hidden_states=False).to(Config.device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [13]:
def create_dataloader(df: pd.DataFrame) -> DataLoader:
    """Create dataloader object with input data."""
    def _create_dataset(_encoded_data: dict) -> TensorDataset:
        """Create dataset object with input data."""
        input_ids = encoded_data['input_ids']
        attention_masks = encoded_data['attention_mask']
        return TensorDataset(input_ids, attention_masks)

    # Tokenize data.
    encoded_data = tokenizer.batch_encode_plus(df.Review.values,
                                               add_special_tokens=Config.add_special_tokens,
                                               return_attention_mask=Config.return_attention_mask,
                                               pad_to_max_length=Config.pad_to_max_length,
                                               max_length=Config.seq_length,
                                               return_tensors=Config.return_tensors)

    # Create dataset object.
    dataset = _create_dataset(encoded_data)

    # Create and return dataloader object.
    return DataLoader(dataset, batch_size=Config.batch_size)

dataloader = create_dataloader(reviews_df)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


## Inference

In [14]:
def infer_predictions(_model: torch.nn.Module, _dataloader: DataLoader) -> np.ndarray:
    """Perform inference using given model on given dataloader."""
    _model.eval()

    y_pred = list()
    for batch in _dataloader:
        batch = tuple(b.to(Config.device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1]
        }

        with torch.no_grad():
            outputs = _model(**inputs)

        probs = torch.nn.functional.softmax(outputs.logits).detach().cpu().numpy()
        y_pred.append(probs)

    y_pred = np.concatenate(y_pred, axis=0)
    return y_pred

In [17]:
raw_predictions = infer_predictions(model, dataloader)
print(classification_report(np.argmax(raw_predictions, axis=1).flatten(), reviews_df.label))

              precision    recall  f1-score   support

           0       0.48      0.20      0.29        54
           1       0.33      0.10      0.15        42
           2       0.03      0.50      0.06         4

    accuracy                           0.17       100
   macro avg       0.28      0.27      0.16       100
weighted avg       0.40      0.17      0.22       100

CPU times: user 19.5 s, sys: 576 ms, total: 20.1 s
Wall time: 11.1 s


## Create custom model wrapper

In [17]:
class CustomWrapper(Model):
    """Custom giskard model wrapper."""
    def model_predict(self, df: pd.DataFrame) -> np.ndarray:
        """Perform inference using overwritten prediction logic."""
        cleaned_df = text_preprocessor(df)
        data_loader = create_dataloader(cleaned_df)
        predicted_probabilities = infer_predictions(self.model, data_loader)
        return predicted_probabilities

In [18]:
wrapped_model = CustomWrapper(model,
                              model_type="classification",
                              classification_labels=[0, 1, 2],
                              name="trip_advisor_sentiment_classifier",
                              feature_names=[TEXT_COLUMN_NAME])

## Validate wrapped model

In [31]:
wrapped_predictions = wrapped_model.predict(wrapped_data)
validation_result = (wrapped_predictions.all_predictions.values == raw_predictions).all()
print(f"Outputs of raw and wrapped model are equal: {validation_result}")

Outputs of raw and wrapped model are equal: True


## Perform scan

In [17]:
%%time
scanning_results = giskard.scan(wrapped_model, wrapped_data)

Your model is successfully validated.
Running scan…
Running detector ModelBiasDetector… 15 issues detected.
Running detector TextPerturbationDetector…Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Hint: "Your tar

In [18]:
display(scanning_results)

<iframe srcdoc="
<script src="https://cdn.tailwindcss.com"></script>
<script>
 tailwind.config = {
 darkMode: 'class'
 }
</script>
<style>
 table.dataframe {
 width: 100%;
 max-width: 100%;
 overflow: auto;
 }

 .dataframe tr {
 border-bottom: 1px solid #555;
 }

 .dataframe td,
 .dataframe th {
 padding: 0.5rem;
 }
</style>



<div class="dark">
 <div id="gsk-scan" class="dark:text-white dark:bg-zinc-900">
 <!-- TAB HEADER -->
<div class="flex items-center items-stretch h-11">
 <div class="flex items-center px-4 dark:fill-white border-b border-gray-500 h-11">
 <svg xmlns="http://www.w3.org/2000/svg" xml:space="preserve" viewBox="0 0 410 213" height="15">
 <path
 d="M318.78 21.94a59.439 59.439 0 0 1 36.45-12.56v.03c7.09-.1 14.13 1.2 20.71 3.84 6.58 2.64 12.57 6.56 17.62 11.54s9.06 10.91 11.8 17.45a53.44 53.44 0 0 1 0 41.3 53.588 53.588 0 0 1-11.8 17.45c-5.05 4.98-11.04 8.9-17.62 11.54s-13.62 3.95-20.71 3.84l-67.92.16-35.87 49.11-135.24 46.84 40.81-47.24a139.229 139.229 0 0 1-66.06-23.8H45.11L0 101.46h55.73C60.38 44.84 115.97 0 183.59 0c31.71 0 63.58 9.86 87.47 27.04a105.149 105.149 0 0 1 26.65 27.18 59.4 59.4 0 0 1 21.07-32.28zM99.85 46.03l55.4 55.45h21.31l-64.81-64.87c-4.18 2.86-8.15 6.01-11.9 9.42zm57.78-28.45 83.9 83.9h21.29l-86.21-86.22c-6.37.32-12.71 1.1-18.98 2.32zm-92.84 98.94H39.72l11.1 9.82h22.44c-3-3.12-5.83-6.4-8.47-9.82zm134.27 0H84.8l-.03.01c20.8 20.91 51.79 33.67 84.66 34.3l29.63-34.31zm-34.81 63.35 77.79-26.94 26.6-36.39h-49.68l-54.71 63.33zm146.77-78.65h46.2l.05.01c9.98-.55 19.36-4.97 26.14-12.32a38.409 38.409 0 0 0 10.16-27.05 38.44 38.44 0 0 0-11.56-26.48 38.405 38.405 0 0 0-26.74-10.94A44.287 44.287 0 0 0 324 37.41a44.268 44.268 0 0 0-12.97 31.28v32.53zm51.82-52.83c5.27.52 9.12 5.22 8.6 10.49-.52 5.27-5.22 9.12-10.49 8.6s-9.12-5.22-8.6-10.49c.53-5.27 5.22-9.12 10.49-8.6z"
 style="fill-rule:evenodd;clip-rule:evenodd" />
 </svg>
 <div class="ml-4 py-2">
 <span class="uppercase text-sm">16 issues detected</span>
 </div>
 </div>

 
 <div data-tab-target="Model_bias" class="h-12 cursor-pointer group active">
 <div
 class="group-[.active]:bg-zinc-800 group-[.active]:border-b-zinc-800 group-[.active]:border-gray-500 border-l px-3 py-2 border-r border-t border-b border-gray-600 border-b-gray-500 h-11">
 Model bias
 
 
 <span class="ml-1 rounded-full text-xs min-w-4 min-h-4 px-1 py-0.5 inline-block text-center bg-red-400">
 
 15
 </span>
 
 </div>
 </div>
 
 <div data-tab-target="Robustness" class="h-12 cursor-pointer group">
 <div
 class="group-[.active]:bg-zinc-800 group-[.active]:border-b-zinc-800 group-[.active]:border-gray-500 border-l px-3 py-2 border-r border-t border-b border-gray-600 border-b-gray-500 h-11">
 Robustness
 
 
 <span class="ml-1 rounded-full text-xs min-w-4 min-h-4 px-1 py-0.5 inline-block text-center bg-red-400">
 
 1
 </span>
 
 </div>
 </div>
 

 <div class="flex-grow border-b border-gray-500 h-11"></div>
</div>
<!-- TAB HEADER END -->
 
<div id="Model_bias" role="tabpanel"
 class="dark:text-white dark:bg-zinc-800 p-4 pt-4 mb-4">
 
 <div class="p-3 mt-2 bg-amber-100/40 rounded-sm w-full flex align-middle">
 <div class="text-amber-100">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="mt-0.5 w-6 h-6">
 <path stroke-linecap="round" stroke-linejoin="round"
 d="M12 9v3.75m-9.303 3.376c-.866 1.5.217 3.374 1.948 3.374h14.71c1.73 0 2.813-1.874 1.948-3.374L13.949 3.378c-.866-1.5-3.032-1.5-3.898 0L2.697 16.126zM12 15.75h.007v.008H12v-.008z" />
 </svg>
 </div>
 <p class="ml-2 my-1 text-amber-100 text-sm">
 We detected some spots in your data where the model has a tendency to make incorrect
 predictions.
 </p>
 </div>
 

 
 <div class="flex items-center space-x-1">
 <h2 class="uppercase my-4 mr-2 font-medium">Issues</h2>
 
 <span class="text-xs border rounded px-1 uppercase text-red-400 border-red-400">15
 major</span>
 
 
 </div>

 <!-- ISSUES TABLE -->
<div class="mb-4">
 <table class="table-auto w-full text-white">
 
 <tbody cl